In [ ]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

# Make matplotlib plot pretty pictures
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 14.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Reload external modules
%load_ext autoreload
%autoreload 2

In [12]:
# Hyper Parameters
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [ ]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(8*8*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [14]:
# CIFAR10 Dataset
train_dataset = dsets.CIFAR10(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=False)

test_dataset = dsets.CIFAR10(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)
        
cnn = CNN()
cnn.cuda()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

Epoch [1/10], Iter [100/500] Loss: 1.4613
Epoch [1/10], Iter [200/500] Loss: 1.3596
Epoch [1/10], Iter [300/500] Loss: 1.2895
Epoch [1/10], Iter [400/500] Loss: 1.3232
Epoch [1/10], Iter [500/500] Loss: 1.0913
Epoch [2/10], Iter [100/500] Loss: 1.2639
Epoch [2/10], Iter [200/500] Loss: 0.8954
Epoch [2/10], Iter [300/500] Loss: 0.9803
Epoch [2/10], Iter [400/500] Loss: 0.8814
Epoch [2/10], Iter [500/500] Loss: 1.1294
Epoch [3/10], Iter [100/500] Loss: 1.0396
Epoch [3/10], Iter [200/500] Loss: 1.0433
Epoch [3/10], Iter [300/500] Loss: 0.8672
Epoch [3/10], Iter [400/500] Loss: 0.9095
Epoch [3/10], Iter [500/500] Loss: 0.9565
Epoch [4/10], Iter [100/500] Loss: 0.7967
Epoch [4/10], Iter [200/500] Loss: 0.8390
Epoch [4/10], Iter [300/500] Loss: 1.0026
Epoch [4/10], Iter [400/500] Loss: 0.7277
Epoch [4/10], Iter [500/500] Loss: 0.9101
Epoch [5/10], Iter [100/500] Loss: 0.8873
Epoch [5/10], Iter [200/500] Loss: 0.8453
Epoch [5/10], Iter [300/500] Loss: 0.8707
Epoch [5/10], Iter [400/500] Loss: